In [27]:
from sklearn import metrics
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn import model_selection

import pandas as pd
import numpy as np
import random as rnd
import os
import re
import itertools
from sklearn.utils import shuffle
import copy
import random

## assist function

In [91]:
from collections import Counter

def main_voting(prob_predict, threshold, weight, voting_method):
    """
    prob_predict: DataFrame, rows: number of instance, columns: number of models
    threshold: float
    weight: list(columns * 1) or None
    voting_method: string, 'hard' or 'soft'
    """
    model_number = len(prob_predict.columns)
    instance_number = len(prob_predict)
    y_pred = []
    y_pred_prob = []
        
    if voting_method == 'hard':
        for row in range(0, instance_number, 1):
            pred_prob = list(prob_predict.iloc[row,])  
            y_pred_all = [int(x > threshold) for x in pred_prob]
            y_pred_hard = Counter(y_pred_all).most_common(1)[0][0]
            y_pred_prob.append(float((Counter(y_pred_all).most_common(1)[0][1]) / model_number))
            y_pred.append(y_pred_hard)
            
    elif voting_method == 'soft':
        if weight == None:
            for row in range(0, instance_number, 1):
                pred_prob = list(prob_predict.iloc[row,])
                y_pred_all = sum(pred_prob) / len(pred_prob)
                y_pred_prob.append(y_pred_all)
                y_pred_soft = int(y_pred_all > threshold)
                y_pred.append(y_pred_soft)
        
        else:
            for row in range(0, instance_number, 1):
                pred_prob = list(prob_predict.iloc[row,])
                y_pred_all = [pred_prob[i] * weight[i] for i in range(0, len(pred_prob), 1)]
                y_pred_prob.append(sum(y_pred_all))
                y_pred_soft_weight = int(sum(y_pred_all) > threshold)
                #y_pred_soft_weight = int((sum(pred_prob) / len(pred_prob)) > threshold)
                y_pred.append(y_pred_soft_weight)
                
    return {'y_pred':y_pred, 'y_pred_prob':y_pred_prob}

def find_all_index(arr,item):
    return [i for i,a in enumerate(arr) if a==item]

def loss_func(pred, true, algo='square'):
    """calculate loss given true label and predicted label

    Args:
      true(int): true label [0, 1]
      prediction(int): predicted lable [0, 1]
      type(string): loss type
    Returns:
      loss value(float)
    """
    if algo == 'square':
        return (true-pred)**2
    elif algo == 'relative-entropy':
        # this will generate error
        return true*np.log(true/pred)+(1-true)*np.log((1-true)/(1-pred))
    elif algo == 'hellinger':
        return 0.5*((np.sqrt(1-true)-np.sqrt(1-pred))**2+(np.sqrt(true)- np.sqrt(pred))**2)
    elif algo == 'absolute':
        return np.abs(true - pred)
    

def decision_prediction_function(pred_labels, weights, nita, c, loss_algo='square', pred_method='mean'):
    """
    """
    vs = weights/np.sum(weights)
    if pred_method=='mean':
        pred_vovk = np.dot(vs, pred_labels)
    elif pred_method=='vovk':
        # this method is not the same as shown in the paper as the results following the formula in the pager doesn't seem right
        delta = lambda x: -c*np.log(np.dot(vs, np.exp(-nita*np.apply_along_axis(loss_func, 0, pred_labels, x, loss_algo))))
        pred_vovk = 0.5*((loss_func(delta(0),0))+(loss_func(delta(1),1)))
    return pred_vovk
    
def tracking_the_best_expert(pred_labels, true_label, weights, alpha, nita, c, method='static-expert', loss_algo='square', pred_method='mean'):
    y_hat = np.int(decision_prediction_function(pred_labels, weights, nita, c, loss_algo, pred_method)>0)
    # loss update
    intermediate_weights = weights*np.exp(-nita*np.apply_along_axis(loss_func, 0, pred_labels, true_label, loss_algo))
#     intermediate_weights = intermediate_weights/np.sum(intermediate_weights)
    n = np.int(pred_labels.shape[0])

    if method == 'static-expert':
        new_weights = intermediate_weights
    elif method == 'fixed-share':
        pool = np.sum(alpha*intermediate_weights)
        new_weights = (1-alpha)*intermediate_weights+(pool - alpha*intermediate_weights)/(n-1)
    elif method == 'variable-share':
        frac = lambda x: (1-alpha)**loss_func(x, true_label, loss_algo)
        temp = np.apply_along_axis(frac, 0, pred_labels)
        pool = np.sum((1-temp)*intermediate_weights)
        new_weights = temp*intermediate_weights+(pool-(1-temp)*intermediate_weights)/(n-1)
        
    new_weights = new_weights/np.sum(new_weights)
#     print('total loss:{}'.format(np.sum(np.apply_along_axis(loss_func, 0, pred_labels, true_label, loss_algo))))
    return new_weights, np.sum(np.apply_along_axis(loss_func, 0, pred_labels, true_label, loss_algo))

## Majority Voting

In [86]:
def majority_voting(data, split_ratio, threshold):
#     result = {}
    data_length = len(data)
    
    model_major_weight = [1] * (len(data.columns) - 1)
    
    train_ratio, validation_ratio, test_ratio =  split_ratio[0], split_ratio[1], split_ratio[2]
    validation_data = data.loc[data_length*train_ratio : data_length*validation_ratio]
    test_data = data.loc[data_length*validation_ratio:]
    
    validation_y = validation_data['y']
    validation_x = validation_data.drop(['y'], axis = 1)
    test_y = test_data['y']
    test_x = test_data.drop(['y'], axis = 1)
    
    all_model_weight = []
    all_model_weight.append(copy.deepcopy(model_major_weight))
    validation_pred = []
    count1 = count0 = 0
    beta = 0.90
    
    for row_num in range(0, len(validation_x), 1):
        x = []
        for i in range(0, len(validation_x.columns), 1):
            x.append(int(validation_x.iloc[row_num,i] > threshold))
        index1 = find_all_index(x, 1)
        index0 = find_all_index(x, 0)
        #print(index0)
        if len(index1) > 0:
            count1 = sum([all_model_weight[-1][i] for i in index1])
        if len(index0) > 0:
            count0 = sum([all_model_weight[-1][i] for i in index0])
        if count0 > count1:
            prediction = 0
        elif count0 < count1:
            prediction = 1
        else:
            prediction = np.random.randint(0,2)
        validation_pred.append(prediction)
        for i in range(0, len(model_major_weight), 1):
            if x[i] != prediction:
                model_major_weight[i] = beta * model_major_weight[i]
        all_model_weight.append(copy.deepcopy([i / sum(model_major_weight) for i in model_major_weight]))
    
    voting = main_voting(test_x, threshold, all_model_weight[-1], 'soft')
    voting['test_y'] = list(test_y)
    voting['final_weight'] = all_model_weight[-1]
    voting['weight_list'] = all_model_weight
    return voting

## random_majority_voting

In [17]:
def random_majority_voting(data, split_ratio, threshold):
    from numpy.random import choice
    
    data_length = len(data)
    model_major_weight = [1] * (len(data.columns) - 1)
    
    train_ratio, validation_ratio, test_ratio =  split_ratio[0], split_ratio[1], split_ratio[2]
    validation_data = data.loc[data_length*train_ratio : data_length*validation_ratio]
    test_data = data.loc[data_length*validation_ratio:]
    
    validation_y = validation_data['y']
    validation_x = validation_data.drop(['y'], axis = 1)
    test_y = test_data['y']
    test_x = test_data.drop(['y'], axis = 1)
    
    beta = 0.90
    model_prediction = []
    all_model_weight1 = []
    all_model_weight1.append(copy.deepcopy(model_major_weight))
    for row_num in range(0, len(validation_x), 1):
        x = []
        choose = [a for a in range(0, len(validation_x.columns), 1)]
        weights = all_model_weight1[-1]
        weights1 = [float(i / sum(all_model_weight1[-1])) for i in all_model_weight1[-1]]
        i = choice(choose, p=weights1)
        model_pred_single = validation_x.iloc[row_num,i]
        if model_pred_single != validation_y.iloc[row_num]:
            weights[i] = beta * weights[i]
        all_model_weight1.append(copy.deepcopy(weights))

    all_model_weight1[-1] = [i / sum(all_model_weight1[-1]) for i in all_model_weight1[-1]]
    voting = main_voting(test_x, threshold, all_model_weight1[-1], 'soft')
    voting['test_y'] = list(test_y)
    voting['final_weight'] = all_model_weight1[-1]
    voting['weight_list'] = all_model_weight1
    return voting 

## perceptron_voting

In [28]:
def perceptron_voting(data, split_ratio, threshold):
    data_length = len(data)
    model_major_weight = [1] * (len(data.columns) - 1)
    
    train_ratio, validation_ratio, test_ratio =  split_ratio[0], split_ratio[1], split_ratio[2]
    validation_data = data.loc[data_length*train_ratio : data_length*validation_ratio]
    test_data = data.loc[data_length*validation_ratio:]
    
    validation_y = validation_data['y']
    validation_x = validation_data.drop(['y'], axis = 1)
    test_y = test_data['y']
    test_x = test_data.drop(['y'], axis = 1)

    for row_num in range(len(validation_x)):
        for i in range(len(validation_x.columns)):
            validation_x.iloc[row_num,i] = int(validation_x.iloc[row_num,i] > 0.5)
    perceptron_df = validation_x
    weight_per = []

    for random_seed in [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]:
        perceptron_clf = Perceptron(tol=1e-3, random_state=random_seed)
        perceptron_clf.fit(perceptron_df, validation_y)
        weight_per.append(perceptron_clf.coef_)
        #y_predict_per = perceptron_clf.predict()
    weight_per1 = list(np.mean(weight_per, axis = 0)[0])
    weight_per2 = [i/sum(weight_per1) for i in weight_per1]
    
    voting = main_voting(test_x, threshold, weight_per2, 'soft')
    voting['test_y'] = list(test_y)
    voting['final_weight'] = weight_per2
    voting['weight_list'] = weight_per2
    return voting 

## Best_Expert using fixed-share chosen function

In [40]:
def best_expert_fixedshare(data, split_ratio, threshold):
    data_length = len(data)
    model_major_weight_track = [1] * (len(data.columns) - 1)
    
    train_ratio, validation_ratio, test_ratio =  split_ratio[0], split_ratio[1], split_ratio[2]
    validation_data = data.loc[data_length*train_ratio : data_length*validation_ratio]
    test_data = data.loc[data_length*validation_ratio:] 
    validation_y = validation_data['y']
    validation_x = validation_data.drop(['y'], axis = 1)
    test_y = test_data['y']
    test_x = test_data.drop(['y'], axis = 1)

    nita = 2
    c = 1/nita
    alpha = 1/1000
    total_loss = 0
    model_prediction = []
    model_number = []
    
    for row_num in range(len(validation_x)):
        for i in range(len(validation_x.columns)):
            validation_x.iloc[row_num,i] = int(validation_x.iloc[row_num,i] > 0.5)
    
    all_model_weight_track = []
    all_model_weight_track.append(copy.deepcopy(model_major_weight_track))
    validation_pred = []
    for row_num in range(0, len(validation_x), 1):
        pred_labels = np.array(validation_x.iloc[row_num,:])
        true_label = validation_y.iloc[row_num]
        model_major_weight_track, loss=tracking_the_best_expert(pred_labels, true_label, model_major_weight_track, alpha, nita, c, 'fixed-share', 'square', 'mean')
        all_model_weight_track.append(copy.deepcopy(list(model_major_weight_track)))
        total_loss += loss
    voting = main_voting(test_x, threshold, all_model_weight_track[-1], 'soft')
    voting['test_y'] = list(test_y)
    voting['final_weight'] = all_model_weight_track[-1]
    voting['weight_list'] = all_model_weight_track
    return voting    

## Best_Expert using variable-share chosen function

In [43]:
def best_expert_variableshare(data, split_ratio, threshold):
    data_length = len(data)
    model_major_weight_track = [1] * (len(data.columns) - 1)
    
    train_ratio, validation_ratio, test_ratio =  split_ratio[0], split_ratio[1], split_ratio[2]
    validation_data = data.loc[data_length*train_ratio : data_length*validation_ratio]
    test_data = data.loc[data_length*validation_ratio:] 
    validation_y = validation_data['y']
    validation_x = validation_data.drop(['y'], axis = 1)
    test_y = test_data['y']
    test_x = test_data.drop(['y'], axis = 1)

    nita = 2
    c = 1/nita
    alpha = 1/1000
    total_loss = 0
    model_prediction = []
    model_number = []
    
    for row_num in range(len(validation_x)):
        for i in range(len(validation_x.columns)):
            validation_x.iloc[row_num,i] = int(validation_x.iloc[row_num,i] > 0.5)
    
    all_model_weight_track = []
    all_model_weight_track.append(copy.deepcopy(model_major_weight_track))
    validation_pred = []
    for row_num in range(0, len(validation_x), 1):
        pred_labels = np.array(validation_x.iloc[row_num,:])
        true_label = validation_y.iloc[row_num]
        model_major_weight_track, loss=tracking_the_best_expert(pred_labels, true_label, model_major_weight_track, alpha, nita, c, 'variable-share', 'square', 'mean')
        all_model_weight_track.append(copy.deepcopy(list(model_major_weight_track)))
        total_loss += loss
    voting = main_voting(test_x, threshold, all_model_weight_track[-1], 'soft')
    voting['test_y'] = list(test_y)
    voting['final_weight'] = all_model_weight_track[-1]
    voting['weight_list'] = all_model_weight_track
    return voting    

## Best_Expert using static-expert chosen function

In [47]:
def best_expert_staticexpert(data, split_ratio, threshold):
    data_length = len(data)
    model_major_weight_track = [1] * (len(data.columns) - 1)
    
    train_ratio, validation_ratio, test_ratio =  split_ratio[0], split_ratio[1], split_ratio[2]
    validation_data = data.loc[data_length*train_ratio : data_length*validation_ratio]
    test_data = data.loc[data_length*validation_ratio:] 
    validation_y = validation_data['y']
    validation_x = validation_data.drop(['y'], axis = 1)
    test_y = test_data['y']
    test_x = test_data.drop(['y'], axis = 1)

    nita = 2
    c = 1/nita
    alpha = 1/1000
    total_loss = 0
    model_prediction = []
    model_number = []
    
    for row_num in range(len(validation_x)):
        for i in range(len(validation_x.columns)):
            validation_x.iloc[row_num,i] = int(validation_x.iloc[row_num,i] > 0.5)
    
    all_model_weight_track = []
    all_model_weight_track.append(copy.deepcopy(model_major_weight_track))
    validation_pred = []
    for row_num in range(0, len(validation_x), 1):
        pred_labels = np.array(validation_x.iloc[row_num,:])
        true_label = validation_y.iloc[row_num]
        model_major_weight_track, loss=tracking_the_best_expert(pred_labels, true_label, model_major_weight_track, alpha, nita, c, 'static-expert', 'square', 'mean')
        all_model_weight_track.append(copy.deepcopy(list(model_major_weight_track)))
        total_loss += loss
    voting = main_voting(test_x, threshold, all_model_weight_track[-1], 'soft')
    voting['test_y'] = list(test_y)
    voting['final_weight'] = all_model_weight_track[-1]
    voting['weight_list'] = all_model_weight_track
    return voting    

In [171]:
x = [[0.6,0.4,0.3,0.4,0],[0.3,0.8,0.9,0.9,1],[0.2,0.9,0.2,0.4,0],[0.2,0.4,0.9,0.4,0],[0.2,0.4,0.2,0.4,0],[0.3,0.8,0.9,0.9,1],[0.3,0.8,0.9,0.9,1],[0.2,0.4,0.2,0.4,0],[0.3,0.8,0.1,0.9,1],[0.3,0.8,0.1,0.9,1],[0.3,0.8,0.1,0.9,1],[0.3,0.8,0.1,0.9,1],[0.3,0.8,0.1,0.9,1],[0.3,0.8,0.1,0.9,1]]
x_test = pd.DataFrame(x, columns = ['1','2','3','4','y'])

In [172]:
print(best_expert_staticexpert(x_test, [0.1,0.5,0.4], 0.5))

{'y_pred': [0, 1, 1, 1, 1, 1, 1], 'y_pred_prob': [0.37615941559557653, 0.7969801711656775, 0.7969801711656775, 0.7969801711656775, 0.7969801711656775, 0.7969801711656775, 0.7969801711656775], 'test_y': [0, 1, 1, 1, 1, 1, 1], 'final_weight': [0.014209336618611042, 0.10499358540350652, 0.10499358540350653, 0.775803492574376], 'weight_list': [[1, 1, 1, 1], [0.3189451556733346, 0.04316453297999626, 0.3189451556733346, 0.3189451556733346], [0.4403985389889412, 0.05960146101105877, 0.05960146101105878, 0.4403985389889412], [0.4403985389889412, 0.05960146101105877, 0.05960146101105878, 0.4403985389889412], [0.09625513525746872, 0.0962551352574687, 0.09625513525746872, 0.7112345942275938], [0.014209336618611042, 0.10499358540350652, 0.10499358540350653, 0.775803492574376], [0.014209336618611042, 0.10499358540350652, 0.10499358540350653, 0.775803492574376]]}


## Classify the function

In [173]:
class majority_voting:
    def __init__(self, threshold = 0.4):
        self.threshold = threshold
    
    def fit(self, X, y):
#         check_classification_targets(y)
        self.classes_ = np.unique(y)
        
        model_major_weight = [1 for cols in range(len(X.columns))]
        all_model_weight = []
        all_model_weight.append(copy.deepcopy(model_major_weight))
        #validation_pred = []
        count1 = count0 = 0
        beta = 0.90
        for row_num in range(0, len(X), 1):
            x1 = []
            for i in range(0, len(X.columns), 1):
                x1.append(int(X.iloc[row_num,i] > self.threshold))
            index1 = find_all_index(x1, 1)
            index0 = find_all_index(x1, 0)
            if len(index1) > 0:
                count1 = sum([all_model_weight[-1][i] for i in index1])
            if len(index0) > 0:
                count0 = sum([all_model_weight[-1][i] for i in index0])
            if count0 > count1:
                prediction = 0
            elif count0 < count1:
                prediction = 1
            else:
                prediction = np.random.randint(0,2)
            #validation_pred.append(prediction)
            print()
            for i in range(0, len(model_major_weight), 1):
                if x1[i] != prediction:
                    model_major_weight[i] = beta * model_major_weight[i]
            all_model_weight.append(copy.deepcopy([i / sum(model_major_weight) for i in model_major_weight]))
        self.weight = all_model_weight[-1]
        self.weight_list = all_model_weight
        
    def predict(self, X):
        model_number = len(X.columns)
        instance_number = len(X)
        y_pred = []
        
        for row in range(0, instance_number, 1):
            pred_prob = list(X.iloc[row,])
            y_pred_all = [pred_prob[i] * self.weight[i] for i in range(0, len(pred_prob), 1)]
            y_pred_soft_weight = int(sum(y_pred_all) > self.threshold)
            y_pred.append(y_pred_soft_weight)       
        return y_pred
    
    def predict_prob(self, X):
        model_number = len(X.columns)
        instance_number = len(X)
        y_pred_prob = []
        for row in range(0, instance_number, 1):
            pred_prob = list(X.iloc[row,])
            y_pred_all = [pred_prob[i] * self.weight[i] for i in range(0, len(pred_prob), 1)]
            y_pred_prob.append([1 - sum(y_pred_all), sum(y_pred_all)])
            
        return y_pred_prob

In [174]:
class random_majority_voting:
    def __init__(self, threshold = 0.4):
        self.threshold = threshold
    
    def fit(self, X, y):
        from numpy.random import choice
        self.classes_ = np.unique(y)
        
        model_major_weight = [1 for cols in range(len(X.columns))]
        all_model_weight = []
        all_model_weight.append(copy.deepcopy(model_major_weight))
        beta = 0.90
        
        for row_num in range(0, len(X), 1):
            x1 = []
            choose = [a for a in range(0, len(X.columns), 1)]
            weights = all_model_weight[-1]
            weights1 = [float(i / sum(all_model_weight[-1])) for i in all_model_weight[-1]]
            i = choice(choose, p=weights1)
            model_pred_single = X.iloc[row_num,i]
            if model_pred_single != validation_y.iloc[row_num]:
                weights[i] = beta * weights[i]
            all_model_weight.append(copy.deepcopy([i / sum(weights) for i in weights]))
        self.weight = all_model_weight[-1]
        self.weight_list = all_model_weight
        
    def predict(self, X):
        model_number = len(X.columns)
        instance_number = len(X)
        y_pred = []
        
        for row in range(0, instance_number, 1):
            pred_prob = list(X.iloc[row,])
            y_pred_all = [pred_prob[i] * self.weight[i] for i in range(0, len(pred_prob), 1)]
            y_pred_soft_weight = int(sum(y_pred_all) > self.threshold)
            y_pred.append(y_pred_soft_weight)       
        return y_pred
    
    def predict_prob(self, X):
        model_number = len(X.columns)
        instance_number = len(X)
        y_pred_prob = []
        for row in range(0, instance_number, 1):
            pred_prob = list(X.iloc[row,])
            y_pred_all = [pred_prob[i] * self.weight[i] for i in range(0, len(pred_prob), 1)]
            y_pred_prob.append([1 - sum(y_pred_all), sum(y_pred_all)])
            
        return y_pred_prob

In [199]:
class perceptron_voting:
    def __init__(self, threshold = 0.4):
        self.threshold = threshold
    
    def fit(self, X, y):
        from numpy.random import choice
        self.classes_ = np.unique(y)
        
        model_major_weight = [1 for cols in range(len(X.columns))]
        all_model_weight = []
        all_model_weight.append(copy.deepcopy(model_major_weight))
        beta = 0.90
        
        for row_num in range(len(X)):
            for i in range(len(X.columns)):
                X.iloc[row_num,i] = int(X.iloc[row_num,i] > self.threshold)
        perceptron_df = X
        weight_per = []
        for random_seed in [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]:
            perceptron_clf = Perceptron(tol=1e-3, random_state=random_seed)
            perceptron_clf.fit(perceptron_df, y)
            print(perceptron_clf.predict(perceptron_df))
            weight_per.append(perceptron_clf.coef_)
        #y_predict_per = perceptron_clf.predict()
        weight_per1 = list(np.mean(weight_per, axis = 0)[0])
        weight_per2 = [i/sum(weight_per1) for i in weight_per1]
    
        self.weight = weight_per2
        self.weight_list = weight_per2
        
    def predict(self, X):
        model_number = len(X.columns)
        instance_number = len(X)
        y_pred = []
        
        for row in range(0, instance_number, 1):
            pred_prob = list(X.iloc[row,])
            y_pred_all = [pred_prob[i] * self.weight[i] for i in range(0, len(pred_prob), 1)]
            y_pred_soft_weight = int(sum(y_pred_all) > self.threshold)
            y_pred.append(y_pred_soft_weight)       
        return y_pred
    
    def predict_prob(self, X):
        import math
        model_number = len(X.columns)
        instance_number = len(X)
        y_pred_prob = []
        for row in range(0, instance_number, 1):
            pred_prob = list(X.iloc[row,])
            y_pred_all = [pred_prob[i] * self.weight[i] for i in range(0, len(pred_prob), 1)]
            prob1 = 1 /(1.1 + math.exp(-sum(y_pred_all)))
            y_pred_prob.append([1 - prob1, prob1])
            
        return y_pred_prob     

In [190]:
class best_expert:
    """
    init:
        algo: 'square','relative-entropy','hellinger','absolute' is used for loss function
        
    """
    def __init__(self, threshold = 0.4, method ='static-expert', loss_algo='square', pred_method='mean'):
        self.threshold = threshold
        self.method = method
        self.loss_algo = loss_algo
        self.pred_method = pred_method
    
    def fit(self, X, y):
        from numpy.random import choice
        self.classes_ = np.unique(y)
        
        model_major_weight = [1 for cols in range(len(X.columns))]
        all_model_weight = []
        all_model_weight.append(copy.deepcopy(model_major_weight))
        nita = 2
        c = 1/nita
        alpha = 1/1000
        total_loss = 0
        
        for row_num in range(len(X)):
            for i in range(len(X.columns)):
                X.iloc[row_num,i] = int(X.iloc[row_num,i] > self.threshold)

        validation_pred = []
        for row_num in range(0, len(X), 1):
            pred_labels = np.array(X.iloc[row_num,:])
            true_label = y.iloc[row_num]
            #print(pred_labels, true_label)
            model_major_weight, loss = self.tracking_the_best_expert(pred_labels, true_label, model_major_weight, alpha, nita, c, self.method, self.loss_algo, self.pred_method)
            all_model_weight.append(copy.deepcopy(list(model_major_weight)))
            total_loss += loss

        self.weight = all_model_weight[-1]
        self.weight_list = all_model_weight
        self.total_loss = total_loss
        
    def predict(self, X):
        model_number = len(X.columns)
        instance_number = len(X)
        y_pred = []
        
        for row in range(0, instance_number, 1):
            pred_prob = list(X.iloc[row,])
            y_pred_all = [pred_prob[i] * self.weight[i] for i in range(0, len(pred_prob), 1)]
            y_pred_soft_weight = int(sum(y_pred_all) > self.threshold)
            y_pred.append(y_pred_soft_weight)       
        return y_pred
    
    def predict_prob(self, X):
        model_number = len(X.columns)
        instance_number = len(X)
        y_pred_prob = []
        for row in range(0, instance_number, 1):
            pred_prob = list(X.iloc[row,])
            y_pred_all = [pred_prob[i] * self.weight[i] for i in range(0, len(pred_prob), 1)]
            y_pred_prob.append([1 - sum(y_pred_all), sum(y_pred_all)])
            
        return y_pred_prob
    
    def loss_func(self, pred, true, algo='square'):
        """calculate loss given true label and predicted label

        Args:
          true(int): true label [0, 1]
          prediction(int): predicted lable [0, 1]
          type(string): loss type
        Returns:
          loss value(float)
        """
        if algo == 'square':
            return (true-pred)**2
        elif algo == 'relative-entropy':
            # this will generate error
            return true*np.log(true/pred)+(1-true)*np.log((1-true)/(1-pred))
        elif algo == 'hellinger':
            return 0.5*((np.sqrt(1-true)-np.sqrt(1-pred))**2+(np.sqrt(true)- np.sqrt(pred))**2)
        elif algo == 'absolute':
            return np.abs(true - pred)
    

    def decision_prediction_function(self, pred_labels, weights, nita, c, loss_algo='square', pred_method='mean'):
        """
        """
        vs = weights/np.sum(weights)
        if pred_method=='mean':
            pred_vovk = np.dot(vs, pred_labels)
        elif pred_method=='vovk':
            # this method is not the same as shown in the paper as the results following the formula in the pager doesn't seem right
            delta = lambda x: -c*np.log(np.dot(vs, np.exp(-nita*np.apply_along_axis(self.loss_func, 0, pred_labels, x, loss_algo))))
            pred_vovk = 0.5*((self.loss_func(delta(0),0))+(self.loss_func(delta(1),1)))
        return pred_vovk

    def tracking_the_best_expert(self, pred_labels, true_label, weights, alpha, nita, c, method='static-expert', loss_algo='square', pred_method='mean'):
        y_hat = np.int(self.decision_prediction_function(pred_labels, weights, nita, c, loss_algo, pred_method)>0)
        # loss update
        intermediate_weights = weights*np.exp(-nita*np.apply_along_axis(self.loss_func, 0, pred_labels, true_label, loss_algo))
    #     intermediate_weights = intermediate_weights/np.sum(intermediate_weights)
        n = np.int(pred_labels.shape[0])

        if method == 'static-expert':
            new_weights = intermediate_weights
        elif method == 'fixed-share':
            pool = np.sum(alpha*intermediate_weights)
            new_weights = (1-alpha)*intermediate_weights+(pool - alpha*intermediate_weights)/(n-1)
        elif method == 'variable-share':
            frac = lambda x: (1-alpha)**self.loss_func(x, true_label, loss_algo)
            temp = np.apply_along_axis(frac, 0, pred_labels)
            pool = np.sum((1-temp)*intermediate_weights)
            new_weights = temp*intermediate_weights+(pool-(1-temp)*intermediate_weights)/(n-1)

        new_weights = new_weights/np.sum(new_weights)
    #     print('total loss:{}'.format(np.sum(np.apply_along_axis(loss_func, 0, pred_labels, true_label, loss_algo))))
        return new_weights, np.sum(np.apply_along_axis(self.loss_func, 0, pred_labels, true_label, loss_algo))
        